In [9]:
import pandas as pd
import numpy as np
import os

In [ ]:
- Extract Data
- Clean Data
- ABC Analysis
      - Helps to businesses prioritize their inventory management 
      - Method - 
            1. Group by Product take aggregations
                  - sum of total value
                  - sum of total total_inventory
                  - first value price_per_unit
                  - sum of 
                        open_stock
                        alocated_inventory
                        damaged_inventory
            2. Calculate Cummulative Sum % of total_inventory
            3. Categorize Based on Cummulative %
                  - A (80%): Products that contribute to the top 80% of the total value.
                  - B (15%): Products that contribute to the next 15% of the total value.
                  - C (5%): Products that contribute to the remaining 5% of the total value.
      - Visualization
            - Pareto Chart
                  - X-axis: Items (ranked from highest to lowest value).
                  - Y-axis: Cumulative percentage of total value.
                  - Bars: Represents the value contribution of each item.
                  - Line: Represents the cumulative percentage.
            - ABC Categories Pie Chart
            - Inventory Value Distribution Histogram
            - ABC Categories Bubble Chart
            - ABC Categories Heatmap

In [31]:
data = pd.read_excel("C:\\Users\\Dell\\Downloads\\Inventory data.xlsx")

In [33]:
data.isna().sum()

batch_no               0
region                 0
warehouse              0
storage_location       0
product_group          0
product_code           0
product_description    0
open_stock             0
alocated_inventory     0
damaged_inventory      0
total_inventory        0
price_per_unit         0
total_price            0
updated_at             0
dtype: int64

In [34]:
def generate_and_save_profile(df, output_file="report.html"):
    profile = ProfileReport(df, title="Pandas Profiling Report")
    profile.to_file(output_file)
    print(f"Profile report saved to {output_file}")

In [35]:
data.dtypes

batch_no                        int64
region                         object
warehouse                      object
storage_location               object
product_group                  object
product_code                   object
product_description            object
open_stock                      int64
alocated_inventory              int64
damaged_inventory               int64
total_inventory                 int64
price_per_unit                  int64
total_price                     int64
updated_at             datetime64[ns]
dtype: object

In [38]:
data.groupby('region').agg({'product_code':'nunique'}).reset_index()

,region,product_code
0,Delhi,80
1,karnataka,80
2,maharashtra,80


In [46]:
data.groupby(['region','warehouse']).agg({'storage_location':['nunique','count'],'product_group':'nunique'}).reset_index()

region  warehouse storage_location       product_group
                                   nunique count       nunique
0        Delhi      Delhi                5   827             4
1    karnataka  Bangalore                5   820             4
2  maharashtra       Pune                5   838             4

In [53]:
#Check - total_inventory = open_stock + alocated_inventory + damaged_inventory
data[~data['total_inventory'] == data[['open_stock','alocated_inventory','damaged_inventory']].sum(axis=1)]

,batch_no,region,warehouse,storage_location,product_group,product_code,product_description,open_stock,alocated_inventory,damaged_inventory,total_inventory,price_per_unit,total_price,updated_at


In [59]:
# check - for each indivisual product code is there any 2 unit prices
(data
    .groupby('product_code')
    .agg({'price_per_unit':'nunique'})
    .reset_index()
    .query("price_per_unit>1"))

,product_code,price_per_unit


In [60]:
data.head(2)

,batch_no,region,warehouse,storage_location,product_group,product_code,product_description,open_stock,alocated_inventory,damaged_inventory,total_inventory,price_per_unit,total_price,updated_at
0,25156,maharashtra,Pune,SL 3,Lighting,Lighting 15,Lighting 15,200,35,10,245,272,66640,2023-08-11
1,25423,maharashtra,Pune,SL 4,Fan,Fan 10,Fan 10,147,10,4,161,107,17227,2023-08-11


In [62]:
# check - total_inventory * price_per_unit == total_price
data[~data['total_price'] == data['total_inventory'] * data['price_per_unit']]

,batch_no,region,warehouse,storage_location,product_group,product_code,product_description,open_stock,alocated_inventory,damaged_inventory,total_inventory,price_per_unit,total_price,updated_at


In [28]:
import plotly.express as px

In [30]:
px.line(result['cumulative_percentage'])

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
result=results
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
result = result.sort_values(by='total_price_sum',ascending=False)

fig.add_trace(go.Bar(x=result['product_code'], 
    y=result['total_price_sum'], name='total_inventory_sum'),
    secondary_y=False,
)


In [110]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
result = result.sort_values(by='total_price_sum',ascending=False)

fig.add_trace(go.Bar(x=result['product_code'], 
    y=result['total_price_sum'], name='total_inventory_sum'),
    secondary_y=False,
)

fig.add_trace(go.Scatter(x=result['product_code'], 
    y=result['cumulative_percentage'], name='cumulative_percentage'),
    secondary_y=True,
)

In [125]:
# Sample data
labels = ['A', 'B', 'C', 'D']
values = [30, 20, 25, 25]

# Create a pie chart
fig = px.pie(
    names=result['ABC_category'].unique(),
    values=pie_chart['count'],
    title='Pie Chart Example',
    hole=0.3  # Add a hole in the center of the pie chart
)
fig.show()

In [131]:
px.histogram(x=result['product_code'],y=result['total_inventory_sum'],color=result['ABC_category'])

In [4]:
import pandas as pd
import plotly.express as px
result = pd.read_excel("../data/presentation/results.xlsx")

In [7]:
result.head(2)

,Unnamed: 0,product_code,open_stock_sum,alocated_inventory_sum,damaged_inventory_sum,total_inventory_sum,total_inventory_std,total_inventory_mean,price_per_unit_first,total_price_sum,total_price_std,total_price_mean,cumulative_percentage,CV,ABC_category,XYZ_category
0,0,Gyeser 13,7406,1284,286,8976,32.057794,187.000000,227,2037552,7277.119137,42449.000000,2.226000,1.59168,A,Z
1,1,Appliance 20,5391,871,167,6429,39.087964,183.685714,267,1716543,10436.486313,49044.085714,4.101301,1.14003,A,Z


In [10]:
result

,Unnamed: 0,product_code,open_stock_sum,alocated_inventory_sum,damaged_inventory_sum,total_inventory_sum,total_inventory_std,total_inventory_mean,price_per_unit_first,total_price_sum,total_price_std,total_price_mean,cumulative_percentage,CV,ABC_category,XYZ_category
0,0,Gyeser 13,7406,1284,286,8976,32.057794,187.000000,227,2037552,7277.119137,42449.000000,2.226000,1.591680,A,Z
1,1,Appliance 20,5391,871,167,6429,39.087964,183.685714,267,1716543,10436.486313,49044.085714,4.101301,1.140030,A,Z
2,2,Appliance 18,6882,1085,231,8198,29.288757,182.177778,208,1705184,6092.061550,37892.977778,5.964193,1.453720,A,Z
3,3,Fan 6,5188,946,207,6341,38.074606,181.171429,265,1680365,10089.770619,48010.428571,7.799970,1.124425,A,Z
4,4,Appliance 2,5660,884,196,6740,33.540617,182.162162,248,1671520,8318.072958,45176.216216,9.626084,1.195179,A,Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,Appliance 14,5074,829,181,6084,36.195547,178.941176,102,620568,3691.945828,18252.000000,97.565050,1.078853,C,Z
76,76,Gyeser 12,4589,725,207,5521,36.936301,178.096774,111,612831,4099.929398,19768.741935,98.234560,0.979018,C,Z
77,77,Fan 10,4764,731,165,5660,30.654825,182.580645,107,605620,3280.066247,19536.129032,98.896192,1.003666,C,Z
78,78,Gyeser 17,3383,511,136,4030,36.757135,175.217391,131,527930,4815.184655,22953.478261,99.472949,0.714625,C,Z


In [15]:
result

,Unnamed: 0,product_code,open_stock_sum,alocated_inventory_sum,damaged_inventory_sum,total_inventory_sum,total_inventory_std,total_inventory_mean,price_per_unit_first,total_price_sum,total_price_std,total_price_mean,cumulative_percentage,CV,ABC_category,XYZ_category
0,0,Gyeser 13,7406,1284,286,8976,32.057794,187.000000,227,2037552,7277.119137,42449.000000,2.226000,1.591680,A,Z
1,1,Appliance 20,5391,871,167,6429,39.087964,183.685714,267,1716543,10436.486313,49044.085714,4.101301,1.140030,A,Z
2,2,Appliance 18,6882,1085,231,8198,29.288757,182.177778,208,1705184,6092.061550,37892.977778,5.964193,1.453720,A,Z
3,3,Fan 6,5188,946,207,6341,38.074606,181.171429,265,1680365,10089.770619,48010.428571,7.799970,1.124425,A,Z
4,4,Appliance 2,5660,884,196,6740,33.540617,182.162162,248,1671520,8318.072958,45176.216216,9.626084,1.195179,A,Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,Appliance 14,5074,829,181,6084,36.195547,178.941176,102,620568,3691.945828,18252.000000,97.565050,1.078853,C,Z
76,76,Gyeser 12,4589,725,207,5521,36.936301,178.096774,111,612831,4099.929398,19768.741935,98.234560,0.979018,C,Z
77,77,Fan 10,4764,731,165,5660,30.654825,182.580645,107,605620,3280.066247,19536.129032,98.896192,1.003666,C,Z
78,78,Gyeser 17,3383,511,136,4030,36.757135,175.217391,131,527930,4815.184655,22953.478261,99.472949,0.714625,C,Z


In [21]:
fig = px.bar(result,x='product_code',y=['open_stock_sum','alocated_inventory_sum','damaged_inventory_sum'],barmode='group')
fig.update_layout(
    title='Inventory Overview by Product Code',
    xaxis_title='Product Code',
    yaxis_title='Quantity'
)